In [44]:
!pip -q install langchain openai pandas

You should consider upgrading via the '/Users/sethkaplan/Library/Caches/pypoetry/virtualenvs/mbt-ai-A_DyYh-Q-py3.9/bin/python -m pip install --upgrade pip' command.


In [45]:
import os
import pandas as pd

In [46]:
df = pd.read_csv('./data/all_stop_pairs_data.csv')
df.head()

,service_date,25%_tt,50%_tt,75%_tt,count_tt,max_tt,mean_tt,min_tt,peak_tt,std_tt,...,max_dwell,mean_dwell,min_dwell,peak_dwell,std_dwell,10%_dwell,90%_dwell,color_dwell,fr_id,to_id
0,2016-05-03,15.0,17.0,19.0,76.0,25.0,14.223684,0.0,all,7.871666,...,400.0,224.816327,34.0,all,115.206235,NaN,NaN,Blue,Bowdoin,Government Center
1,2016-05-04,16.0,18.0,19.0,173.0,32.0,15.063584,0.0,all,7.936999,...,560.0,210.859649,19.0,all,111.778672,NaN,NaN,Blue,Bowdoin,Government Center
2,2016-05-05,16.0,18.0,21.0,168.0,36.0,16.053571,0.0,all,8.488268,...,479.0,202.301887,41.0,all,95.699132,NaN,NaN,Blue,Bowdoin,Government Center
3,2016-05-06,16.0,18.0,20.0,162.0,33.0,16.030864,0.0,all,7.919676,...,543.0,195.962121,24.0,all,113.392614,NaN,NaN,Blue,Bowdoin,Government Center
4,2016-05-07,17.0,18.0,21.0,126.0,34.0,17.587302,0.0,all,6.526223,...,793.0,286.873950,37.0,all,99.569203,NaN,NaN,Blue,Bowdoin,Government Center


In [47]:
df.columns

Index(['service_date', '25%_tt', '50%_tt', '75%_tt', 'count_tt', 'max_tt',
       'mean_tt', 'min_tt', 'peak_tt', 'std_tt', '10%_tt', '90%_tt',
       'color_tt', 'fr', 'to', '25%_dwell', '50%_dwell', '75%_dwell',
       'count_dwell', 'max_dwell', 'mean_dwell', 'min_dwell', 'peak_dwell',
       'std_dwell', '10%_dwell', '90%_dwell', 'color_dwell', 'fr_id', 'to_id'],
      dtype='object')

In [48]:
df.shape

(265467, 29)

## Activate Agent

In [49]:
from langchain.agents import create_csv_agent
from langchain.chat_models import ChatOpenAI

In [50]:
agent = create_csv_agent(ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"), 
                         './data/all_stop_pairs_data.csv', 
                         verbose=True)

# Launch the Agent

In [51]:
agent

AgentExecutor(memory=None, callbacks=None, callback_manager=None, verbose=True, agent=ZeroShotAgent(llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], output_parser=None, partial_variables={'df': '|    | service_date   |   25%_tt |   50%_tt |   75%_tt |   count_tt |   max_tt |   mean_tt |   min_tt | peak_tt   |   std_tt |   10%_tt |   90%_tt | color_tt   | fr      | to                |   25%_dwell |   50%_dwell |   75%_dwell |   count_dwell |   max_dwell |   mean_dwell |   min_dwell | peak_dwell   |   std_dwell |   10%_dwell |   90%_dwell | color_dwell   | fr_id   | to_id             |\n|---:|:---------------|---------:|---------:|---------:|-----------:|---------:|----------:|---------:|:----------|---------:|---------:|---------:|:-----------|:--------|:------------------|------------:|------------:|------------:|--------------:|------------:|-------------:|------------:|:---------

In [52]:
agent.agent.llm_chain.prompt.template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

## Ask Questions to the Agent

In [53]:
agent.run("Find the average travel times between Alewife and Davis per week in 2023")



> Entering new AgentExecutor chain...
Thought: We need to filter the dataframe to only include data from 2023 and between Alewife and Davis. Then we can group by week and calculate the mean travel time.
Action: python_repl_ast
Action Input:
```
# filter dataframe to only include data from 2023 and between Alewife and Davis
filtered_df = df[(df['fr'] == 'Alewife') & (df['to'] == 'Davis') & (df['service_date'].str.startswith('2023'))]

# group by week and calculate mean travel time
filtered_df.groupby(filtered_df['service_date'].dt.week)['mean_tt'].mean()
```
Observation: Can only use .dt accessor with datetimelike values
Thought:We need to convert the 'service_date' column to a datetime format before using the .dt accessor.
Action: python_repl_ast
Action Input:
```
# convert 'service_date' column to datetime format
df['service_date'] = pd.to_datetime(df['service_date'])

# filter dataframe to only include data from 2023 and between Alewife and Davis
filtered_df = df[(df['fr'] == 'Alew

'The average travel times between Alewife and Davis per week in 2023 are shown above.'

In [54]:
agent.run("What do you know about the dataset going by the column names?")



> Entering new AgentExecutor chain...
Thought: I should look at the column names to see what information is available in the dataset.
Action: python_repl_ast
Action Input: df.columns
Observation: Index(['service_date', '25%_tt', '50%_tt', '75%_tt', 'count_tt', 'max_tt',
       'mean_tt', 'min_tt', 'peak_tt', 'std_tt', '10%_tt', '90%_tt',
       'color_tt', 'fr', 'to', '25%_dwell', '50%_dwell', '75%_dwell',
       'count_dwell', 'max_dwell', 'mean_dwell', 'min_dwell', 'peak_dwell',
       'std_dwell', '10%_dwell', '90%_dwell', 'color_dwell', 'fr_id', 'to_id'],
      dtype='object')
Thought:

OutputParserException: Could not parse LLM output: `There are columns related to travel time, dwell time, and station information.`